In [ ]:
## EDA

In [1]:
from tqdm import tqdm
import pandas as pd
import json
import string
import nltk
from nltk.corpus import stopwords
from  nltk import FreqDist
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import numpy as np

In [3]:
data = [json.loads(line) for line in open('reviews_Electronics_5.json', 'r')]

In [21]:
df = pd.DataFrame(data)

In [22]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200
1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400
3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1290556800
4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1317254400


In [23]:
df.shape

(1689188, 9)

In [25]:
df['overall'].value_counts()

5.0    1009026
4.0     347041
3.0     142257
1.0     108725
2.0      82139
Name: overall, dtype: int64

In [26]:
# drop reviews to reduce class imabalance and reduce size of data
remove_n_5 = 900000
remove_n_4 = 247000
remove_n_3 = 40000

just5 = df.loc[df['overall'] == 5]
just4 = df.loc[df['overall'] == 4]
just3 = df.loc[df['overall'] == 3]

drop_indices_5 = np.random.choice(just5.index, remove_n_5, replace=False)
drop_indices_4 = np.random.choice(just4.index, remove_n_4, replace=False)
drop_indices_3 = np.random.choice(just3.index, remove_n_3, replace=False)

df_final = df.drop(drop_indices_5)
df_final = df_final.drop(drop_indices_4)
df_final = df_final.drop(drop_indices_3)

In [27]:
df_final['overall'].value_counts()

5.0    109026
1.0    108725
3.0    102257
4.0    100041
2.0     82139
Name: overall, dtype: int64

In [28]:
df_final.reset_index(drop=True,inplace=True)

In [29]:
df_text = df_final[['asin','reviewText','overall','reviewerID']]

In [30]:
df_text['asin_reviewer'] = df_text['asin'].astype(str)+ '_' + df_text['reviewerID'].astype(str)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_text.drop(columns=['asin','reviewerID'],axis=1,inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [32]:
df_text['review_length'] = df_text['reviewText'].apply(len)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
g = sns.FacetGrid(data=df_text, col='overall')
g.map(plt.hist, 'review_length', bins=30)
g.set(xlim=(0,10000))

In [ ]:
sns.boxplot(x='overall', y='review_length', data=df_text)

# NLP

In [33]:
# expand list of stopwords
new_stopwords = [x.replace("'","") for x in stopwords.words('english')]
final_stopwords = set(new_stopwords+stopwords.words('english')+list(string.punctuation))

In [155]:
## cleaning text
def clean_review(review):
    clean = []
    joined_clean_review = ''
    for x in review:
        if x in string.punctuation:
            x = x.replace(x, " ")
        elif x not in string.punctuation:
            x = x.lower()
        clean.append(x)    
        joined_clean_review = "".join(clean)

    return joined_clean_review

In [156]:
def get_tokens(clean_review):
    #  tokenize & remove stop words
    list_of_tokens = [x for x in word_tokenize(clean_review) if x not in final_stopwords]
    
    # return list of text from each review
    return list_of_tokens

In [157]:
# function to get lemmas
def lem_words(list_of_tokens,lemmatizer):
    wrd_list = [lemmatizer.lemmatize(word) for word in list_of_tokens]
    return " ".join(wrd_list)

In [158]:
def finalize_token(reviews):
    corpus = []
    for review in tqdm(reviews):
        clean = clean_review(review)
        tokens = get_tokens(clean)
        lemmas = lem_words(tokens,lemmatizer)
        corpus.append(lemmas)
    return corpus

In [39]:
# take out review text from dataframe and make it a series?
reviews = df_text['reviewText']

In [145]:
reviews[0]

'I\'m a professional OTR truck driver, and I bought a TND 700 at a truck stop hoping to make my life easier.  Rand McNally, are you listening?First thing I did after charging it was connect it to my laptop and install the software and then attempt to update it.  The software detected a problem with my update and wanted my home address so I could be sent a patch on an SD card.  Hello?  I don\'t think I\'m all that unusual; my home address is a PO box that a friend checks weekly and that I might get to check every six months or so.  I live in my truck and at truck stops.  If you need to make a patch available on an SD card then you should send the SD cards to the truck stops where the devices are sold.  I ran the update program multiple times until the program said that the TND 700 was completely updated.I programmed in the height (13\'6"), the length (53\') and the weight (80,000#) of my rig and told it that I preferred highways.  I was parked at a truck stop in the Cincinnati OH area. 

In [ ]:
corpus = finalize_token(reviews)






  0%|          | 0/502188 [00:00<?, ?it/s]




  0%|          | 2/502188 [00:00<13:06:37, 10.64it/s]




  0%|          | 17/502188 [00:00<9:27:32, 14.75it/s]




  0%|          | 44/502188 [00:00<6:46:37, 20.58it/s]




  0%|          | 67/502188 [00:00<4:57:07, 28.16it/s]




  0%|          | 81/502188 [00:01<6:49:34, 20.43it/s]




  0%|          | 92/502188 [00:01<6:01:48, 23.13it/s]




  0%|          | 101/502188 [00:02<5:29:41, 25.38it/s]




  0%|          | 108/502188 [00:02<4:42:37, 29.61it/s]




  0%|          | 115/502188 [00:02<4:45:08, 29.35it/s]




  0%|          | 121/502188 [00:02<5:24:29, 25.79it/s]




  0%|          | 126/502188 [00:03<5:22:57, 25.91it/s]




  0%|          | 130/502188 [00:03<6:38:40, 20.99it/s]




  0%|          | 138/502188 [00:03<5:14:00, 26.65it/s]




  0%|          | 143/502188 [00:03<5:09:24, 27.04it/s]




  0%|          | 150/502188 [00:03<4:16:15, 32.65it/s]




  0%|          | 156/502188 [00:03<3:44:30, 37.27it/s]




  0%|     

  0%|          | 1867/502188 [00:18<1:01:01, 136.64it/s]




  0%|          | 1898/502188 [00:18<51:01, 163.42it/s]  




  0%|          | 1919/502188 [00:18<49:47, 167.44it/s]




  0%|          | 1939/502188 [00:18<55:56, 149.02it/s]




  0%|          | 1957/502188 [00:19<55:10, 151.11it/s]




  0%|          | 1983/502188 [00:19<51:56, 160.48it/s]




  0%|          | 2001/502188 [00:19<1:01:15, 136.10it/s]




  0%|          | 2017/502188 [00:19<59:34, 139.92it/s]  




  0%|          | 2033/502188 [00:19<1:22:35, 100.93it/s]




  0%|          | 2049/502188 [00:19<1:17:29, 107.57it/s]




  0%|          | 2062/502188 [00:20<1:34:44, 87.97it/s] 




  0%|          | 2073/502188 [00:20<1:49:37, 76.04it/s]




  0%|          | 2083/502188 [00:20<1:42:57, 80.96it/s]




  0%|          | 2094/502188 [00:20<1:38:10, 84.90it/s]




  0%|          | 2114/502188 [00:20<1:21:13, 102.61it/s]




  0%|          | 2132/502188 [00:20<1:13:30, 113.38it/s]




  0%|          | 2150/502188 [00:20

  1%|          | 4063/502188 [00:34<40:41, 203.99it/s]




  1%|          | 4106/502188 [00:34<34:53, 237.88it/s]




  1%|          | 4141/502188 [00:34<31:56, 259.86it/s]




  1%|          | 4176/502188 [00:34<29:29, 281.51it/s]




  1%|          | 4237/502188 [00:34<24:47, 334.71it/s]




  1%|          | 4299/502188 [00:34<21:23, 387.77it/s]




  1%|          | 4346/502188 [00:35<21:13, 390.84it/s]




  1%|          | 4398/502188 [00:35<19:39, 422.02it/s]




  1%|          | 4445/502188 [00:35<22:14, 372.95it/s]




  1%|          | 4490/502188 [00:35<21:13, 390.89it/s]




  1%|          | 4548/502188 [00:35<19:10, 432.69it/s]




  1%|          | 4595/502188 [00:35<22:54, 361.89it/s]




  1%|          | 4636/502188 [00:35<32:15, 257.08it/s]




  1%|          | 4670/502188 [00:36<48:11, 172.07it/s]




  1%|          | 4704/502188 [00:36<43:38, 190.00it/s]




  1%|          | 4736/502188 [00:36<39:18, 210.92it/s]




  1%|          | 4782/502188 [00:36<33:26, 247.85it/s]



  1%|▏         | 6595/502188 [00:52<43:08, 191.43it/s]




  1%|▏         | 6616/502188 [00:52<53:54, 153.19it/s]




  1%|▏         | 6637/502188 [00:52<50:04, 164.96it/s]




  1%|▏         | 6668/502188 [00:52<43:14, 191.00it/s]




  1%|▏         | 6698/502188 [00:52<38:51, 212.53it/s]




  1%|▏         | 6736/502188 [00:53<34:19, 240.62it/s]




  1%|▏         | 6764/502188 [00:53<48:40, 169.63it/s]




  1%|▏         | 6787/502188 [00:53<58:39, 140.78it/s]




  1%|▏         | 6806/502188 [00:53<1:18:06, 105.71it/s]




  1%|▏         | 6826/502188 [00:53<1:07:51, 121.67it/s]




  1%|▏         | 6843/502188 [00:54<1:07:21, 122.58it/s]




  1%|▏         | 6859/502188 [00:54<1:03:28, 130.07it/s]




  1%|▏         | 6875/502188 [00:54<1:17:24, 106.64it/s]




  1%|▏         | 6888/502188 [00:54<1:17:51, 106.02it/s]




  1%|▏         | 6913/502188 [00:54<1:05:03, 126.89it/s]




  1%|▏         | 6929/502188 [00:54<1:12:40, 113.57it/s]




  1%|▏         | 6943/502188 [00:55<1:52

  2%|▏         | 8695/502188 [01:09<1:21:33, 100.85it/s]




  2%|▏         | 8706/502188 [01:09<1:33:20, 88.12it/s] 




  2%|▏         | 8724/502188 [01:09<1:19:44, 103.13it/s]




  2%|▏         | 8741/502188 [01:09<1:11:09, 115.56it/s]




  2%|▏         | 8755/502188 [01:09<1:09:25, 118.47it/s]




  2%|▏         | 8790/502188 [01:09<56:16, 146.15it/s]  




  2%|▏         | 8809/502188 [01:09<1:02:33, 131.43it/s]




  2%|▏         | 8826/502188 [01:10<1:13:10, 112.36it/s]




  2%|▏         | 8841/502188 [01:10<1:08:15, 120.47it/s]




  2%|▏         | 8860/502188 [01:10<1:00:48, 135.21it/s]




  2%|▏         | 8884/502188 [01:10<52:50, 155.58it/s]  




  2%|▏         | 8903/502188 [01:10<1:00:12, 136.54it/s]




  2%|▏         | 8925/502188 [01:10<54:39, 150.39it/s]  




  2%|▏         | 8942/502188 [01:11<1:08:04, 120.76it/s]




  2%|▏         | 8957/502188 [01:11<1:24:02, 97.81it/s] 




  2%|▏         | 8970/502188 [01:11<3:10:55, 43.05it/s]




  2%|▏         | 8980/502

...

 83%|████████▎ | 419133/502188 [52:50<07:28, 185.22it/s]




 83%|████████▎ | 419171/502188 [52:50<06:21, 217.41it/s]




 83%|████████▎ | 419197/502188 [52:50<06:18, 219.48it/s]




 83%|████████▎ | 419244/502188 [52:50<05:17, 261.04it/s]




 83%|████████▎ | 419276/502188 [52:50<05:37, 245.60it/s]




 83%|████████▎ | 419317/502188 [52:50<04:57, 278.46it/s]




 84%|████████▎ | 419350/502188 [52:50<05:54, 233.66it/s]




 84%|████████▎ | 419378/502188 [52:51<19:48, 69.69it/s] 




 84%|████████▎ | 419405/502188 [52:52<15:47, 87.39it/s]




 84%|████████▎ | 419426/502188 [52:52<13:08, 104.93it/s]




 84%|████████▎ | 419447/502188 [52:52<11:48, 116.80it/s]




 84%|████████▎ | 419492/502188 [52:52<09:11, 149.98it/s]




 84%|████████▎ | 419523/502188 [52:52<07:47, 176.97it/s]




 84%|████████▎ | 419565/502188 [52:52<06:26, 213.77it/s]




 84%|████████▎ | 419610/502188 [52:52<05:26, 253.15it/s]




 84%|████████▎ | 419647/502188 [52:52<04:55, 279.46it/s]




 84%|████████▎ | 419708/5

 84%|████████▍ | 422306/502188 [53:09<12:41, 104.88it/s]




 84%|████████▍ | 422333/502188 [53:09<10:22, 128.18it/s]




 84%|████████▍ | 422390/502188 [53:09<07:58, 166.81it/s]




 84%|████████▍ | 422443/502188 [53:10<06:25, 206.87it/s]




 84%|████████▍ | 422496/502188 [53:10<05:14, 253.16it/s]




 84%|████████▍ | 422540/502188 [53:10<04:45, 279.40it/s]




 84%|████████▍ | 422592/502188 [53:10<04:05, 324.41it/s]




 84%|████████▍ | 422648/502188 [53:10<03:35, 369.77it/s]




 84%|████████▍ | 422695/502188 [53:10<04:16, 309.58it/s]




 84%|████████▍ | 422745/502188 [53:10<03:49, 346.27it/s]




 84%|████████▍ | 422787/502188 [53:10<03:54, 338.22it/s]




 84%|████████▍ | 422826/502188 [53:11<10:23, 127.25it/s]




 84%|████████▍ | 422855/502188 [53:12<16:38, 79.44it/s] 




 84%|████████▍ | 422884/502188 [53:12<13:19, 99.19it/s]




 84%|████████▍ | 422934/502188 [53:12<10:45, 122.73it/s]




 84%|████████▍ | 422970/502188 [53:12<08:39, 152.58it/s]




 84%|████████▍ | 423013/5

 85%|████████▍ | 425664/502188 [53:30<25:57, 49.12it/s]




 85%|████████▍ | 425675/502188 [53:30<21:40, 58.85it/s]




 85%|████████▍ | 425708/502188 [53:30<16:26, 77.54it/s]




 85%|████████▍ | 425764/502188 [53:30<12:11, 104.41it/s]




 85%|████████▍ | 425791/502188 [53:30<10:16, 123.85it/s]




 85%|████████▍ | 425840/502188 [53:30<08:08, 156.19it/s]




 85%|████████▍ | 425880/502188 [53:30<06:39, 190.97it/s]




 85%|████████▍ | 425913/502188 [53:30<06:21, 199.82it/s]




 85%|████████▍ | 425946/502188 [53:31<05:55, 214.35it/s]




 85%|████████▍ | 425975/502188 [53:31<07:09, 177.39it/s]




 85%|████████▍ | 425999/502188 [53:31<07:35, 167.41it/s]




 85%|████████▍ | 426021/502188 [53:31<07:44, 164.13it/s]




 85%|████████▍ | 426046/502188 [53:31<07:05, 178.80it/s]




 85%|████████▍ | 426067/502188 [53:31<06:48, 186.28it/s]




 85%|████████▍ | 426090/502188 [53:31<06:45, 187.65it/s]




 85%|████████▍ | 426132/502188 [53:32<05:39, 224.18it/s]




 85%|████████▍ | 426159/502

 85%|████████▌ | 428522/502188 [53:48<09:17, 132.25it/s]




 85%|████████▌ | 428560/502188 [53:48<07:27, 164.36it/s]




 85%|████████▌ | 428595/502188 [53:48<06:22, 192.21it/s]




 85%|████████▌ | 428621/502188 [53:48<05:54, 207.26it/s]




 85%|████████▌ | 428647/502188 [53:49<09:43, 125.98it/s]




 85%|████████▌ | 428667/502188 [53:49<13:04, 93.67it/s] 




 85%|████████▌ | 428688/502188 [53:49<10:59, 111.37it/s]




 85%|████████▌ | 428712/502188 [53:49<09:14, 132.56it/s]




 85%|████████▌ | 428732/502188 [53:50<08:34, 142.79it/s]




 85%|████████▌ | 428771/502188 [53:50<06:56, 176.13it/s]




 85%|████████▌ | 428796/502188 [53:50<07:29, 163.18it/s]




 85%|████████▌ | 428825/502188 [53:50<06:30, 187.77it/s]




 85%|████████▌ | 428849/502188 [53:50<11:50, 103.19it/s]




 85%|████████▌ | 428868/502188 [53:51<12:14, 99.88it/s] 




 85%|████████▌ | 428890/502188 [53:51<10:31, 116.08it/s]




 85%|████████▌ | 428913/502188 [53:51<09:12, 132.71it/s]




 85%|████████▌ | 428931/

 86%|████████▌ | 431099/502188 [54:11<09:07, 129.91it/s]




 86%|████████▌ | 431127/502188 [54:11<07:42, 153.68it/s]




 86%|████████▌ | 431164/502188 [54:12<06:21, 186.27it/s]




 86%|████████▌ | 431188/502188 [54:12<07:59, 148.15it/s]




 86%|████████▌ | 431208/502188 [54:12<08:40, 136.29it/s]




 86%|████████▌ | 431244/502188 [54:12<07:20, 161.21it/s]




 86%|████████▌ | 431265/502188 [54:12<07:11, 164.23it/s]




 86%|████████▌ | 431293/502188 [54:12<06:29, 182.03it/s]




 86%|████████▌ | 431314/502188 [54:12<06:22, 185.17it/s]




 86%|████████▌ | 431359/502188 [54:13<05:15, 224.84it/s]




 86%|████████▌ | 431387/502188 [54:13<04:58, 237.48it/s]




 86%|████████▌ | 431415/502188 [54:13<07:36, 154.94it/s]




 86%|████████▌ | 431437/502188 [54:13<07:39, 153.96it/s]




 86%|████████▌ | 431474/502188 [54:13<06:25, 183.42it/s]




 86%|████████▌ | 431498/502188 [54:13<08:40, 135.69it/s]




 86%|████████▌ | 431518/502188 [54:14<09:17, 126.72it/s]




 86%|████████▌ | 431551/

In [ ]:
import csv

myFile = open('corpus.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(corpus)

In [ ]:
myFile = open('lemmatized_reviews.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(lemmatized_reviews)

In [ ]:
def top_200_tokens(tokens):
    freqdist = FreqDist(tokens)
    # get the 200 most common words 
    most_common = freqdist.most_common(200)
    return most_common

In [ ]:
## vectorize with count & tf-idf
cv = CountVectorizer()
cv.fit(lemmatized_reviews)

In [ ]:
vector = cv.transform(lemmatized_reviews)